# Part 6 -- Combine Features and Target (AAPL)

Prepare data for **Singular Value Decomposition (SVD)**.

### Load lib codes:

In [17]:
!pwd

/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter


In [18]:
from os import chdir
chdir('/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/')

from lib import *
# suppress_warnings()
from lib.twitter_keys import my_keys

In [39]:
AAPL_df = pd.read_pickle('../predicting_stock_market_trends_with_Twitter/data/AAPL_df.pickle')

In [20]:
X_tfidf = pd.read_pickle('../predicting_stock_market_trends_with_Twitter/data/X_tfidf.pickle')

In [21]:
tweets_df = pd.read_pickle('../predicting_stock_market_trends_with_Twitter/data/tweets_df')

### Pull Y's based on date, drop non-shared, then vectorize cleaned_text

In [40]:
AAPL_df.tail()

,Date,Open,High,Low,Close,Volume,Diff,Diff_Encoded
2365,2017-05-25,153.73,154.35,153.03,153.87,19235598,0.53,1.0
2366,2017-05-26,154.00,154.24,153.31,153.61,21927637,-0.26,-1.0
2367,2017-05-30,153.42,154.43,153.33,153.67,20126851,0.06,1.0
2368,2017-05-31,153.97,154.17,152.38,152.76,24451164,-0.91,-1.0
2369,2017-06-01,153.17,153.33,152.22,153.18,16404088,0.42,1.0


In [22]:
tweets_df.head()

,_id,text,timestamp,username,cleaned_text,Date
0,593592642c7311008077a8a5,"On this National Gun Violence Awareness Day, l...",2017-06-02 17:35:54,BarackObama,On this National Gun Violence Awareness Day le...,2017-06-02
1,593592642c7311008077a8a6,Forever grateful for the service and sacrifice...,2017-05-29 13:09:16,BarackObama,Forever grateful for the service and sacrifice...,2017-05-29
2,593592642c7311008077a8a7,Good to see my friend Prince Harry in London t...,2017-05-27 13:15:25,BarackObama,Good to see my friend Prince Harry in London t...,2017-05-27
3,593592642c7311008077a8a8,"Through faith, love, and resolve the character...",2017-05-25 14:13:35,BarackObama,Through faith love and resolve the character o...,2017-05-25
4,593592642c7311008077a8a9,Our hearts go out to those killed and wounded ...,2017-05-23 16:56:14,BarackObama,Our hearts go out to those killed and wounded ...,2017-05-23


In [12]:
len(tweets_df['Date'].unique())

2578

In [16]:
min(tweets_df['Date'].unique())

datetime.date(2008, 9, 10)

In [23]:
len(AAPL_df['Date'].unique())

2370

In [24]:
len(tweets_df)

87289

In [37]:
tweets_df['Date'][3]

datetime.date(2017, 5, 25)

In [41]:
AAPL_df['Date'][2365]

datetime.date(2017, 5, 25)

In [42]:
tweets_df['Date'][3]==AAPL_df['Date'][2365]

True

In [58]:
[AAPL_df['Date']]

[0       2008-01-02
 1       2008-01-03
 2       2008-01-04
 3       2008-01-07
 4       2008-01-08
 5       2008-01-09
 6       2008-01-10
 7       2008-01-11
 8       2008-01-14
 9       2008-01-15
 10      2008-01-16
 11      2008-01-17
 12      2008-01-18
 13      2008-01-22
 14      2008-01-23
 15      2008-01-24
 16      2008-01-25
 17      2008-01-28
 18      2008-01-29
 19      2008-01-30
 20      2008-01-31
 21      2008-02-01
 22      2008-02-04
 23      2008-02-05
 24      2008-02-06
 25      2008-02-07
 26      2008-02-08
 27      2008-02-11
 28      2008-02-12
 29      2008-02-13
            ...    
 2340    2017-04-20
 2341    2017-04-21
 2342    2017-04-24
 2343    2017-04-25
 2344    2017-04-26
 2345    2017-04-27
 2346    2017-04-28
 2347    2017-05-01
 2348    2017-05-02
 2349    2017-05-03
 2350    2017-05-04
 2351    2017-05-05
 2352    2017-05-08
 2353    2017-05-09
 2354    2017-05-10
 2355    2017-05-11
 2356    2017-05-12
 2357    2017-05-15
 2358    2017-05-16


In [77]:
for i in tweets_df['text'][:10]:
    print('yes')

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


In [79]:
data = []
for i in tweets_df['text'][:10]:
    print(i)
#     if tweets_df['Date'] in list(AAPL_df['Date']):
#         data.append('yes')
#     else:
#         data.append('no')

On this National Gun Violence Awareness Day, let your voice be heard and show your commitment to reducing gun viole… https://t.co/S7dth6Ywb0
Forever grateful for the service and sacrifice of all who fought to protect our freedoms and defend this country we… https://t.co/BwRS0onlOL
Good to see my friend Prince Harry in London to discuss the work of our foundations &amp; offer condolences to victims… https://t.co/f1U8It8AmE
Through faith, love, and resolve the character of the people of Manchester has shown itself.  Our thoughts &amp; prayer… https://t.co/xyMer891Zo
Our hearts go out to those killed and wounded in Manchester. Americans will always stand shoulder to shoulder with the people of the UK.
Excited to hear from Sierra, Imani, Filiz, and Benedikt who are making a difference at home and abroad. See you in… https://t.co/bF1kB1GvxC
Happy Mother's Day to my love and partner on this journey @MichelleObama, and to all the wonderful, hardworking mothers out there.
We're rolling up our 

In [80]:
def match_dates_and_pull_data(features_df, target_df):
    data = []
    for i in tweets_df['Date']:
        try:
            if i in list(target_df['Date']):
                data.append(target_df['Diff_Encoded'])
            else:
                data.append('No Match')
        except:
            pass
    print(data)

In [82]:
start = datetime.now()

match_dates_and_pull_data(tweets_df, AAPL_df)

end = datetime.now()
print(end - start)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [50]:
tweets_df

,_id,text,timestamp,username,cleaned_text,Date
0,593592642c7311008077a8a5,"On this National Gun Violence Awareness Day, l...",2017-06-02 17:35:54,BarackObama,On this National Gun Violence Awareness Day le...,2017-06-02
1,593592642c7311008077a8a6,Forever grateful for the service and sacrifice...,2017-05-29 13:09:16,BarackObama,Forever grateful for the service and sacrifice...,2017-05-29
2,593592642c7311008077a8a7,Good to see my friend Prince Harry in London t...,2017-05-27 13:15:25,BarackObama,Good to see my friend Prince Harry in London t...,2017-05-27
3,593592642c7311008077a8a8,"Through faith, love, and resolve the character...",2017-05-25 14:13:35,BarackObama,Through faith love and resolve the character o...,2017-05-25
4,593592642c7311008077a8a9,Our hearts go out to those killed and wounded ...,2017-05-23 16:56:14,BarackObama,Our hearts go out to those killed and wounded ...,2017-05-23
5,593592642c7311008077a8aa,"Excited to hear from Sierra, Imani, Filiz, and...",2017-05-22 21:16:23,BarackObama,Excited to hear from Sierra Imani Filiz and Be...,2017-05-22
6,593592642c7311008077a8ab,Happy Mother's Day to my love and partner on t...,2017-05-14 14:01:16,BarackObama,Happy Mother s Day to my love and partner on t...,2017-05-14
7,593592642c7311008077a8ac,"We're rolling up our sleeves again, back where...",2017-05-03 19:42:16,BarackObama,We re rolling up our sleeves again back where ...,2017-05-03
8,593592642c7311008077a8ad,"Well said, Jimmy. That's exactly why we fought...",2017-05-02 17:19:26,BarackObama,Well said Jimmy That s exactly why we fought s...,2017-05-02
9,593592642c7311008077a8ae,RT @ObamaFoundation: It’s a beautiful day on t...,2017-04-24 19:48:17,BarackObama,RT ObamaFoundation It s a beautiful day on the...,2017-04-24


In [36]:
naive_vectorizer = CountVectorizer(min_df = 3)
document_term_matrix = create_document_term_matrix(naive_vectorizer, tweets_df['text'])
document_term_matrix.sample(5)
#this is a document term matrix. It just counts the # of times a word shows up:

NameError: name 'CountVectorizer' is not defined

### Do a PCA to determine number of components for SVD: